In [ ]:
import sys
from pathlib import Path

# Assumption: CWD == .../usv-autopilot/analysis/sims/notebooks
REPO_ROOT = Path.cwd().parents[2]

# Sanity check (fail fast if something changes)
if not (REPO_ROOT / "docs" / "index.md").exists():
    raise RuntimeError(f"Repo root assumption failed. REPO_ROOT={REPO_ROOT}")

# Make repo importable
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("cwd:", Path.cwd())
print("repo root:", REPO_ROOT)

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt

# Add repo root to sys.path so we can import analysis.sims...
REPO_ROOT = Path().resolve().parents[3]  # .../analysis/sims/notebooks -> repo root
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from usv_sim.digital_twin.process_model import ProcessParams
from analysis.sims.scenarios.step import make_step_us
from analysis.sims.scenarios.circle import make_constant_turn
from analysis.sims.scenarios.zigzag import make_zigzag_ud

# The simulator entry point
from usv_sim.digital_twin.current import simulate, simulate_with_inputs

In [ ]:
# Common sim settings
# NOTE: U = [u_s, u_d] is interpreted as effective/achieved actuation in the process model.
dt = 0.02
T = 60.0

# State order: [x, y, psi, v, r, b_g]
x0 = np.zeros(6, dtype=np.float64)

# Placeholder params (tune later)
params = ProcessParams(
    tau_v=2.0,
    tau_r=0.8,
    k_v=0.8,
    k_r=1.2,
)

In [ ]:
scenarios = [
    make_step_us(dt=dt, T=T, u_s0=0.0, u_s1=0.4, t_step=5.0, u_d=0.0, x0=x0, params=params),
    make_constant_turn(dt=dt, T=T, u_s=0.35, u_d=0.15, x0=x0, params=params),
    make_zigzag_ud(dt=dt, T=T, u_s=0.35, u_d_amp=0.2, period=6.0, x0=x0, params=params),
]

In [ ]:
results = {}

plt.figure()
for sc in scenarios:
    t, X, U = simulate_with_inputs(sc.x0, sc.U, sc.dt, sc.params)
    # X shape: (N+1, 6), where N = sc.U.shape[0]
    results[sc.name] = (sc, X)
    plt.plot(X[:, 0], X[:, 1], label=sc.name)

plt.axis("equal")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("XY paths (open-loop inputs)")
plt.legend()
plt.show()

In [ ]:
name = scenarios[2].name  # pick which
sc, X = results[name]
t = np.arange(X.shape[0]) * sc.dt

# Speed v(t)
plt.figure()
plt.plot(t, X[:, 3])
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.title(f"v(t) — {name}")
plt.show()

# Heading psi(t)
plt.figure()
plt.plot(t, X[:, 2])
plt.xlabel("t [s]")
plt.ylabel("psi [rad]")
plt.title(f"psi(t) — {name}")
plt.show()